In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import csv
from requests import post
import numpy as np

In [2]:
steam_data =pd.read_csv(r"C:\Users\valdi\OneDrive\Documents\Springboard\steam.csv")
steam_data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
steam_data.shape

(27075, 18)

In [4]:
steam_data.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [5]:
#The next step will be dropping data that is unneeded for the machine learning program
#which are "appid","required_age", "english", "achievements", "owners", "steamspy_tags",  and "price"
# as these columns either do not reflect traits of trending games or are redundant.  

In [6]:
steam_data=steam_data.drop(['appid','english','required_age','achievements','owners','price','steamspy_tags' ], axis=1)
steam_data.columns

Index(['name', 'release_date', 'developer', 'publisher', 'platforms',
       'categories', 'genres', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime'],
      dtype='object')

In [ ]:
COLUMN DESCRIPTION:
name-tilte of the game
release_date- when the game was released
developer-developer of the game
publisher-publisher of the game
platforms- what platforms these games were on
categories- determines the type of game
genres- determines the sub-type of the game 
positive_ratings- ratings that reflect positve reviews of the game 
negative_ratings- rating that reflect negative reviews of the game
average_playtime- the average amount of time a player spent on the game
median_playtime- the midpoint of this dataset on playtime

In [17]:
steam_data.head()

,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,average_playtime,median_playtime
0,Counter-Strike,2000-11-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,124534,3339,17612,317
1,Team Fortress Classic,1999-04-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,3318,633,277,62
2,Day of Defeat,2003-05-01,Valve,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,3416,398,187,34
3,Deathmatch Classic,2001-06-01,Valve,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,1273,267,258,184
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,5250,288,624,415


In [23]:
steam_data.describe(include = 'all')

,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,average_playtime,median_playtime
count,27075,27075,27075,27075,27075,27075,27075,2.707500e+04,27075.000000,27075.000000,27075.00000
unique,27033,2619,17113,14354,7,3333,1552,NaN,NaN,NaN,NaN
top,Dark Matter,2018-07-13,Choice of Games,Big Fish Games,windows,Single-player,Action;Indie,NaN,NaN,NaN,NaN
freq,3,64,94,212,18398,6110,1852,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000559e+03,211.027147,149.804949,146.05603
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.898872e+04,4284.938531,1827.038141,2353.88008
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.00000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000e+00,2.000000,0.000000,0.00000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000e+01,9.000000,0.000000,0.00000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.260000e+02,42.000000,0.000000,0.00000


In [ ]:
#the reason why NaN seen so often in first few colmuns is due to the fact that 
#columns "name", 'release_date;, 'devleoper', 'publisher', 'platforms', 'categories'
# and 'genres' are considered to be categorical data 

In [14]:
print(f'name: {steam_data.name.dtype}')
print(f'release_date: {steam_data.release_date.dtype}')
print(f'developer: {steam_data.developer.dtype}')
print(f'publisher: {steam_data.publisher.dtype}')
print(f'platforms: {steam_data.platforms.dtype}')
print(f'categories: {steam_data.categories.dtype}')
print(f'genres: {steam_data.genres.dtype}')
print(f'positive_ratings: {steam_data.positive_ratings.dtype}')
print(f'negative_ratings: {steam_data.negative_ratings.dtype}')
print(f'average_playtime: {steam_data.average_playtime.dtype}')
print(f'median_playtime: {steam_data.median_playtime.dtype}')

name: object
release_date: object
developer: object
publisher: object
platforms: object
categories: object
genres: object
positive_ratings: int64
negative_ratings: int64
average_playtime: int64
median_playtime: int64


In [24]:
print(f'name: {steam_data.name.nunique()}')
print(f'release_date: {steam_data.release_date.nunique()}')
print(f'developer: {steam_data.developer.nunique()}')
print(f'publisher: {steam_data.publisher.nunique()}')
print(f'platforms: {steam_data.platforms.nunique()}')
print(f'categories: {steam_data.categories.nunique()}')
print(f'genres: {steam_data.genres.nunique()}')
print(f'positive_ratings: {steam_data.positive_ratings.nunique()}')
print(f'negative_ratings: {steam_data.negative_ratings.nunique()}')
print(f'average_playtime: {steam_data.average_playtime.nunique()}')
print(f'median_playtime: {steam_data.median_playtime.nunique()}')
#This is done so the output may reflect on how the data was measured as well as noticing which colmuns may have
#repeating/missing rows

name: 27033
release_date: 2619
developer: 17113
publisher: 14354
platforms: 7
categories: 3333
genres: 1552
positive_ratings: 2800
negative_ratings: 1492
average_playtime: 1345
median_playtime: 1312


In [25]:
steam_data.isnull().sum()

name                0
release_date        0
developer           0
publisher           0
platforms           0
categories          0
genres              0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
dtype: int64

In [26]:
duplicate=steam_data[steam_data.duplicated()]
print(duplicate)
#An empty dataframe is produced, meaning that there are no values in this dataframe, therfore it is dafe to conclude there are 
#no duplicated values

Empty DataFrame
Columns: [name, release_date, developer, publisher, platforms, categories, genres, positive_ratings, negative_ratings, average_playtime, median_playtime]
Index: []


In [ ]:
steam_data['positive_rating']